#Netcat

This notebook provide some basic netcat functionality which you need to let a stream run so that the Spark Streaming Engine can process it.

The simplest way is to click `Run All` on the top. (You can change the message it sends when you want). <br>
To do this, after you run it, you have to `stop the Execution` and you have to manually execute `nc.close()`. <br>
Then you can modify the function `send(nc)` and then press `Run All` again.

In [ ]:
import socket
import time

class Netcat:

    """ Python 'netcat like' module """

    def __init__(self, ip, port):

        self.buff = ""
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.socket.bind((ip, port))
        self.socket.listen(10)
        self.conn, addr = self.socket.accept()
 
    def write(self, data):
        # append newline between messages
        data = data + "\n"
        # encode before sending through socket
        message = data.encode()
        self.conn.send(message)
    
    def close(self):

      self.socket.close()

In [ ]:
def send(nc):
  # RUN ONLY 3 iterations per each window of 1 second to make sure that wordcount works as expected
  i = 0
  while True:
    try:
      print("Sending iteration "+ str(i))
      nc.write('Hello World Bye World')
      nc.write('Hello Spark Goodbye Spark')
      nc.write("ass isn't allowed.")
      i+=1
      if i == 3:
        time.sleep(1)
        i = 0
    except Exception as e:
      print(str(e))
      break

In [ ]:
# start a new Netcat() instance
# Note: When you experiment with the producer you might receive the following error message: OSError: [Errno 98] Address already in use
#       A simple fix is to change the port number of the producer and the consumer, e.g. from 9999 to 9998
while True:
  nc = Netcat("127.0.0.1", 9999)
  try:
    send(nc)
  except Exception as e:
    print(str(e))
  finally:
    nc.close()


In [ ]:
# Always close at the end
nc.close()